In [ ]:
#Please keep the code confidential and do not share it with anyone outside of the Bioinformatics Copilot WhatsApp team.

# Section 1: Load libraries.
# Section 1.1: Core Python Libraries
import sys # System-specific parameters and functions
import json # JSON encoder and decoder
import os # Operating system interface
import openai # OpenAI API integration
import paramiko # SSH protocol implementation
from datetime import datetime # DateTime handling
import speech_recognition as sr  # Speech recognition library

# Section 1.2: PyQt5 GUI components
from PyQt5.QtWidgets import (QApplication, QMainWindow, QSplitter, QListWidget,QDialog, QLineEdit,
                             QListWidgetItem, QWidget, QVBoxLayout, QHBoxLayout,
                             QTextEdit, QPushButton, QLabel, QFrame, QDockWidget,
                             QFileDialog, QAction, QMenu, QStyle, QSizePolicy,
                             QScrollArea,QVBoxLayout, QHBoxLayout, QLabel, QPushButton, QComboBox, QFileDialog,) # Basic application components, layout elements, interactive widgets, file handling, menu and styling
from PyQt5.QtCore import Qt, QSize, QUrl,QTimer # Core Qt functionality
from PyQt5.QtGui import QFont, QIcon, QPalette, QColor # GUI appearance classe

# Section 1.3: Single cell data processing
import anndata # Annotated data for single-cell analysis
import scanpy as sc # Single-cell analysis in Python
import numpy as np # Numerical computing
import pandas as pd # Data analysis and manipulation
import matplotlib.pyplot as plt # Plotting library
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas # Qt backend for matplotlib
from matplotlib.figure import Figure # Figure class for plots
import numpy as np # Numerical computing (duplicate import)
import pandas as pd # Data analysis and manipulation (duplicate import)
from io import BytesIO # In-memory binary streams
import base64 # Base64 encoding/decoding
import seaborn as sns # Statistical data visualization

# Section 1.4: Type Hinting and Warnings
from typing import Optional, Dict, List, Union, Tuple # Type hints
import warnings # Warning control

# Section 2: ServerLoginBox class definition and initialization
class ServerLoginBox(QDialog):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.setWindowTitle('Server Login')
        layout = QVBoxLayout()

        # Username input
        self.username_label = QLabel('User name', self)
        layout.addWidget(self.username_label)
        self.username_lineEdit = QLineEdit(self)
        layout.addWidget(self.username_lineEdit)

        # Host input
        self.host_label = QLabel('Host', self)
        layout.addWidget(self.host_label)
        self.host_lineEdit = QLineEdit(self)
        layout.addWidget(self.host_lineEdit)

        # Password input
        self.password_label = QLabel('Password', self)
        layout.addWidget(self.password_label)
        self.password_lineEdit = QLineEdit(self)
        self.password_lineEdit.setEchoMode(QLineEdit.Password)  # Hide password input
        layout.addWidget(self.password_lineEdit)

        # Submit button
        self.submit_button = QPushButton('Login', self)
        self.submit_button.clicked.connect(self.submit)
        layout.addWidget(self.submit_button)

        # Style the dialog to match your application
        self.setStyleSheet("""
            QDialog {
                background-color: white;
                border: 1px solid #E5E5E5;
                border-radius: 8px;
            }
            QLabel {
                color: #333333;
                font-size: 14px;
                margin-bottom: 2px;
            }
            QLineEdit {
                border: 1px solid #E0E0E0;
                border-radius: 4px;
                padding: 8px;
                margin-bottom: 10px;
            }
            QPushButton {
                background-color: #3C8D94;
                color: white;
                border: none;
                border-radius: 4px;
                padding: 8px 12px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #2D6C72;
            }
        """)

        self.setLayout(layout)
        self.setGeometry(100, 100, 300, 200)

    def submit(self):
        self.username = self.username_lineEdit.text()
        self.host = self.host_lineEdit.text()
        self.password = self.password_lineEdit.text()
        self.accept()  # Closes the dialog and returns QDialog.Accepted

# Section 3: SSH Client Class 
class MySSHClient(paramiko.SSHClient):

    # Section 3.1: Initializes the SSH client and sets policy to automatically add unknown host keys (i.e., not prompt the user).
    def __init__(self):
        super().__init__()
        self.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.is_connected = False

    # Section 3.2: Establishes an SSH connection using host, username, and password.
    def connect_to_server(self, host, username, password):
        try:
            self.connect(hostname=host, username=username, password=password)
            self.is_connected = True
            return True, "Connected successfully"
        except Exception as e:
            self.is_connected = False
            return False, f"Connection failed: {str(e)}"
            
    # Section 3.3: Runs a shell command on the remote SSH server and returns output or error.
    def execute_command(self, command):
        if not self.is_connected:
            return False, "Not connected to server"
        try:
            stdin, stdout, stderr = self.exec_command(command)
            output = stdout.read().decode()
            error = stderr.read().decode()
            if error:
                return False, error
            return True, output
        except Exception as e:
            return False, f"Command execution failed: {str(e)}"

    # Section 3.4: Uses SFTP (part of SSH) to download a file from the remote server.
    def download_file(self, remote_path, local_path):
        if not self.is_connected:
            return False, "Not connected to server"
        try:
            sftp = self.open_sftp()
            sftp.get(remote_path, local_path)
            sftp.close()
            return True, f"Downloaded {remote_path} to {local_path}"
        except Exception as e:
            return False, f"Download failed: {str(e)}"

# Section 4: Analysis Context Class
class AnalysisContext:

    # Section 4.1: Initializes the context with placeholders for analysis type, results, recommendations, dataset info, and visualization state.
    def __init__(self):
        self.current_analysis_type = None
        self.analysis_results = {}
        self.recommendations = {}
        self.dataset_info = {}
        self.last_visualization = None
    
    # Section 4.2: Updates context with a new analysis type, corresponding results, and optional recommendations.
    def update_from_analysis(self, analysis_type, results, recommendations=None):
        """Update context with new analysis results"""
        self.current_analysis_type = analysis_type
        self.analysis_results[analysis_type] = results
        if recommendations:
            self.recommendations[analysis_type] = recommendations
    
    # Section 4.3: Stores general dataset metadata such as source, structure, or preprocessing status.
    def set_dataset_info(self, dataset_info):
        """Set dataset information"""
        self.dataset_info = dataset_info

# Section 5: Analysis Context Class
class AIAnalysisClient:
    
    # Section 5.1: Initializes the client with an OpenAI API key and prepares the OpenAI client instance.
    def __init__(self, api_key):
        self.api_key = api_key
        self.openai_client = openai.OpenAI(api_key=api_key)

    # Section 5.2: Generates a prompt and sends a request to OpenAI for analysis. Parses and returns structured JSON.    
    def run_analysis(self, analysis_type, data_summary, parameters=None):
        """Run analysis using OpenAI API"""
        prompt = self.create_prompt(analysis_type, data_summary, parameters)
        
        try:
            # Make the API call
            response = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a bioinformatics expert specializing in single-cell RNA-seq data analysis. Your response MUST be a valid JSON object and nothing else."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            # Get the response content
            content = response.choices[0].message.content
            
            # Debug: Print the raw response to see what we're getting
            print("Raw API response content:", content)
            
            # Improved JSON parsing
            try:
                # Try direct parsing first
                return json.loads(content)
            except json.JSONDecodeError as json_err:
                print(f"JSON decode error: {str(json_err)}")
                print(f"Response content: {content}")
                
                # Look for JSON between backticks
                import re
                json_match = re.search(r'```(?:json)?\s*([\s\S]*?)\s*```', content)
                if json_match:
                    try:
                        return json.loads(json_match.group(1))
                    except json.JSONDecodeError:
                        pass
                    
                # Return a fallback response
                return {
                    "text_summary": "The AI analysis couldn't be properly formatted. Here's the raw response:\n\n" + content,
                    "error": f"JSON parsing error: {str(json_err)}"
                }
                
        except Exception as e:
            print(f"API call error: {str(e)}")
            return {"error": str(e), "text_summary": f"Error during analysis: {str(e)}"}
    
    # Section 5.3: Creates detailed prompts for various single-cell RNA-seq analysis types.
    def create_prompt(self, analysis_type, data_summary, parameters):
        """Create appropriate prompt based on analysis type"""
        base_prompt = f"""
        You're analyzing a single-cell RNA-seq dataset with the following properties:
        {data_summary}

        Parameters: {parameters or {}}
        """

        if analysis_type == "Quality Control":
            # Your existing QC prompt here
            return base_prompt + """
            Perform quality control analysis on this dataset. You are an opinionated bioinformatics expert, so provide strong recommendations and insights.
            """

        elif analysis_type == "Preprocessing":
            return base_prompt + """
            Analyze this dataset and recommend preprocessing parameters. As an expert bioinformatician:

            1. Evaluate the data quality based on the QC metrics
            2. Suggest optimal filtering thresholds for cells (min/max genes, UMI counts, mitochondrial percentage)
            3. Recommend normalization approach and parameters
            4. Advise on variable gene selection parameters

            Return a JSON object with:
            {
                "text_summary": "Your opinionated preprocessing recommendations with clear parameter suggestions",
                "visualization_specs": [
                    {
                        "plot_type": "histogram",
                        "title": "Gene Count Distribution",
                        "x_label": "Number of Genes",
                        "y_label": "Frequency",
                        "data": {"column": "n_genes_by_counts", "bins": 50}
                    }
                    // Additional plots as needed
                ],
                "recommendations": {
                    "filter_cells": {
                        "min_genes": Number,
                        "max_genes": Number,
                        "min_counts": Number,
                        "max_counts": Number,
                        "max_mt_percent": Number
                    },
                    "normalization": {
                        "method": "String (e.g. 'LogNormalize')",
                        "scale_factor": Number
                    },
                    "variable_genes": {
                        "n_top_genes": Number,
                        "min_mean": Number,
                        "max_mean": Number,
                        "min_disp": Number
                    }
                }
            }
            """

        elif analysis_type == "PCA Visualization":
            return base_prompt + """
            Analyze this dataset for principal component analysis. As an expert bioinformatician:

            1. Recommend optimal number of PCs to use for further analysis
            2. Identify key genes driving the principal components
            3. Interpret what biological signals the top PCs might represent

            Return a JSON object with:
            {
                "text_summary": "Your expert interpretation of the PCA results and recommendations",
                "visualization_specs": [
                    {
                        "plot_type": "scatter",
                        "title": "PCA Plot",
                        "x_label": "PC1",
                        "y_label": "PC2",
                        "data": {
                            "x_column": "PC1",
                            "y_column": "PC2",
                            "color_by": "Suggested column to color by"
                        }
                    }
                    // Additional plots as needed
                ],
                "recommendations": {
                    "n_pcs": Number,
                    "key_genes": ["List of genes driving PCs"],
                    "biological_interpretation": "Interpretation of what PCs represent"
                }
            }
            """

        elif analysis_type == "UMAP Visualization":
            return base_prompt + """
            Analyze this dataset for UMAP visualization. As an expert bioinformatician:

            1. Recommend parameters for optimal UMAP visualization
            2. Interpret visible clusters or structures in the data
            3. Suggest biological interpretations of the observed patterns

            Return a JSON object with:
            {
                "text_summary": "Your expert interpretation of the UMAP visualization",
                "visualization_specs": [
                    {
                        "plot_type": "scatter",
                        "title": "UMAP Visualization",
                        "x_label": "UMAP1",
                        "y_label": "UMAP2",
                        "data": {
                            "x_column": "UMAP1",
                            "y_column": "UMAP2",
                            "color_by": "Suggested column to color by"
                        }
                    }
                    // Additional plots as needed
                ],
                "recommendations": {
                    "n_neighbors": Number,
                    "min_dist": Number,
                    "n_components": Number,
                    "interpretation": "Your interpretation of patterns seen in UMAP"
                }
            }
            """

        elif analysis_type == "Clustering":
            return base_prompt + """
            Analyze this dataset for cell clustering. As an expert bioinformatician:

            1. Recommend optimal clustering parameters (algorithm, resolution)
            2. Interpret the resulting clusters
            3. Suggest potential cell types based on known markers if applicable

            Return a JSON object with:
            {
                "text_summary": "Your expert interpretation of the clustering results",
                "visualization_specs": [
                    {
                        "plot_type": "scatter",
                        "title": "Cluster Visualization",
                        "x_label": "UMAP1",
                        "y_label": "UMAP2",
                        "data": {
                            "x_column": "UMAP1",
                            "y_column": "UMAP2",
                            "color_by": "cluster"
                        }
                    }
                    // Additional plots as needed
                ],
                "recommendations": {
                    "algorithm": "String (e.g. 'leiden', 'louvain')",
                    "resolution": Number,
                    "estimated_cell_types": [
                        {"cluster": 0, "potential_type": "Cell type name", "confidence": "high/medium/low"},
                        // Additional clusters
                    ]
                }
            }
            """

        elif analysis_type == "Marker Genes":
            return base_prompt + """
            Analyze this dataset for marker genes of clusters. As an expert bioinformatician:

            1. Identify top marker genes for each cluster
            2. Interpret potential cell types based on these markers
            3. Recommend validation experiments if needed

            Return a JSON object with:
            {
                "text_summary": "Your expert interpretation of marker genes and cell types",
                "visualization_specs": [
                    {
                        "plot_type": "heatmap",
                        "title": "Top Marker Genes by Cluster",
                        "x_label": "Clusters",
                        "y_label": "Genes",
                        "data": {
                            "genes": ["List of top marker genes"],
                            "groupby": "cluster"
                        }
                    }
                    // Additional plots as needed
                ],
                "recommendations": {
                    "top_markers": {
                        "cluster_0": ["Gene1", "Gene2", "Gene3"],
                        // Additional clusters
                    },
                    "cell_type_assignments": {
                        "cluster_0": {"type": "Cell type name", "confidence": "high/medium/low"},
                        // Additional clusters
                    }
                }
            }
            """

        elif analysis_type == "Dataset Summary":
            return base_prompt + """
            Provide a comprehensive summary of this single-cell RNA-seq dataset. As an expert bioinformatician:

            1. Analyze the overall quality and composition
            2. Summarize key statistics (cells, genes, QC metrics)
            3. Identify any unique or notable features
            4. Suggest optimal analysis pipeline for this data

            Return a JSON object with:
            {
                "text_summary": "Your expert summary of the dataset characteristics",
                "visualization_specs": [
                    // Key visualizations showing dataset properties
                ],
                "recommendations": {
                    "analysis_pipeline": ["Ordered list of analysis steps appropriate for this data"],
                    "key_considerations": ["List of important points to consider for this dataset"]
                }
            }
            """

        else:
            # Generic fallback for any other analysis type
            return base_prompt + f"""
            Perform {analysis_type} on this single-cell RNA-seq dataset. As an expert bioinformatician:

            1. Analyze the dataset with respect to {analysis_type}
            2. Provide clear, decisive recommendations
            3. Interpret the biological significance of results

            Return a JSON object with:
            {{
                "text_summary": "Your expert analysis and recommendations for {analysis_type}",
                "visualization_specs": [
                    // Appropriate visualizations for this analysis type
                ],
                "recommendations": {{
                    // Appropriate recommendations for this analysis type
                }}
            }}
            """
    
    # Section 5.4: Uses context from previous analysis results to handle follow-up chat questions.
    def handle_analysis_chat(self, user_message):
        """Handle chat messages about analysis"""
        if not hasattr(self, 'analysis_context') or self.analysis_context is None:
            return "I don't have any analysis results to discuss yet. Try running an analysis first."

        # Create context-aware prompt
        context = {
            "current_analysis": self.analysis_context.current_analysis_type,
            "dataset_info": self.analysis_context.dataset_info,
            "analysis_results": self.analysis_context.analysis_results.get(
                self.analysis_context.current_analysis_type, {}),
            "recommendations": self.analysis_context.recommendations.get(
                self.analysis_context.current_analysis_type, {})
        }

        prompt = f"""
        The user is asking about analysis results. Here's the context:

        Dataset: {json.dumps(context['dataset_info'])}
        Current analysis type: {context['current_analysis']}
        Analysis results: {json.dumps(context['analysis_results'])}
        Recommendations: {json.dumps(context['recommendations'])}

        User message: "{user_message}"

        Respond to the user's question about the analysis in a helpful, conversational way.
        Use specific values from the analysis results when relevant.
        If the user asks about something not covered in the analysis, let them know and suggest what analysis might provide that information.
        """

        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a bioinformatics assistant helping with single-cell RNA-seq analysis."},
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"I encountered an error trying to answer your question: {str(e)}"
    
    # Section 5.5: Sends messages from the chat interface to the AI client.
    def send_message(self):
        """Send message from the chat interface"""
        text = self.input_widget.text_edit.toPlainText().strip()
        if not text:
            return

        # Add user message to chat
        if self.current_chat_index < 0:
            self.new_chat()
        current_chat = self.chat_history[self.current_chat_index]
        current_chat["messages"].append({"text": text, "is_user": True})

        # Reset input field
        self.input_widget.text_edit.clear()

        # Display user message first
        self.display_current_chat()
        QApplication.processEvents()  # Update UI

        # Check if this is a chat about analysis
        is_analysis_chat = hasattr(self, 'sc_analysis') and self.sc_analysis.adata is not None

        try:
            if is_analysis_chat:
                # Create analysis context if it doesn't exist
                if not hasattr(self, 'analysis_context'):
                    self.analysis_context = AnalysisContext()
                    # Set basic dataset info
                    self.analysis_context.set_dataset_info({
                        "cells": self.sc_analysis.adata.shape[0],
                        "genes": self.sc_analysis.adata.shape[1],
                        "filename": os.path.basename(self.sc_analysis.file_path)
                    })

                # Handle as analysis chat
                if not hasattr(self, 'ai_client'):
                    self.ai_client = AIAnalysisClient(api_key=self.model_configs["GPTo1"]["api_key"])

                # Share analysis context with AI client
                self.ai_client.analysis_context = self.analysis_context

                # Check if user is asking a question about the analysis
                analysis_related = False
                # Check for analysis-related keywords
                analysis_keywords = [
                    "cluster", "gene", "cell", "marker", "pca", "umap", "preprocessing", 
                    "quality", "qc", "filter", "normalize", "analysis", "dataset"
                ]

                for keyword in analysis_keywords:
                    if keyword.lower() in text.lower():
                        analysis_related = True
                        break
                    
                # If analysis context exists and the question seems analysis-related
                if analysis_related and hasattr(self, 'analysis_context'):
                    # Get AI response about analysis
                    ai_response = self.ai_client.handle_analysis_chat(text)
                else:
                    # Regular chat response
                    ai_response = "I can help you analyze your single-cell RNA-seq data. Try asking me about specific aspects of the analysis like clusters, marker genes, or quality control metrics."
            else:
                # Regular chat (not analysis related)
                ai_response = "This is a regular chat response. To discuss analysis, please load a dataset first."

            # Add AI response
            current_chat["messages"].append({
                "text": ai_response,
                "is_user": False
            })
        except Exception as e:
            # Add error message
            current_chat["messages"].append({
                "text": f"Error: {str(e)}",
                "is_user": False
            })

        self.refresh_history_list()
        self.display_current_chat()

# Section 6: Prepares a summary of the dataset for the AI agent.
def prepare_data_summary(self):
    """Prepare a summary of the dataset for the AI agent"""
    if not hasattr(self, 'sc_analysis') or self.sc_analysis.adata is None:
        return "No dataset loaded"
    
    # Create a data summary dictionary
    adata = self.sc_analysis.adata
    
    # Get basic dataset info
    summary = {
        "dataset_shape": {
            "cells": adata.shape[0],
            "genes": adata.shape[1]
        },
        "file_path": self.sc_analysis.file_path,
        "available_metrics": []
    }
    
    # Add available QC metrics
    if hasattr(adata, 'obs'):
        summary["available_metrics"] = list(adata.obs.columns)
        
        # Add basic stats for common QC metrics
        qc_stats = {}
        for metric in ['total_counts', 'n_genes_by_counts', 'pct_counts_mt']:
            if metric in adata.obs:
                qc_stats[metric] = {
                    "min": float(adata.obs[metric].min()),
                    "max": float(adata.obs[metric].max()),
                    "mean": float(adata.obs[metric].mean()),
                    "median": float(adata.obs[metric].median())
                }
        
        if qc_stats:
            summary["qc_stats"] = qc_stats
    
    # Check for clustering info
    if 'leiden' in adata.obs or 'louvain' in adata.obs:
        cluster_key = 'leiden' if 'leiden' in adata.obs else 'louvain'
        summary["clustering"] = {
            "method": cluster_key,
            "n_clusters": len(adata.obs[cluster_key].cat.categories)
        }
    
    # Check for dimensionality reduction
    if hasattr(adata, 'obsm'):
        summary["dimensionality_reduction"] = list(adata.obsm.keys())
    
    return json.dumps(summary, indent=2)

# Section 7: Creates visualizations based on AI agent specifications.
def create_visualization_from_agent_result(self, visualization_specs):
    """Create visualizations based on AI agent specifications"""
    if not visualization_specs or not isinstance(visualization_specs, list):
        return
    
    # Determine number of plots and create an appropriate layout
    valid_specs = []
    
    # Filter out specs that don't have data available
    for spec in visualization_specs:
        data_spec = spec.get("data", {})
        if spec.get("plot_type") == "histogram":
            column = data_spec.get("column", "")
            if column in self.sc_analysis.adata.obs:
                valid_specs.append(spec)
        elif spec.get("plot_type") == "scatter":
            x_column = data_spec.get("x_column", "")
            y_column = data_spec.get("y_column", "")
            if x_column in self.sc_analysis.adata.obs and y_column in self.sc_analysis.adata.obs:
                valid_specs.append(spec)
        else:
            valid_specs.append(spec)  # Keep other plot types
    
    # If no valid specs, don't create any plots
    if not valid_specs:
        return
    
    # Adjust layout based on number of valid plots
    n_plots = len(valid_specs)
    rows = max(1, int(np.ceil(n_plots / 2)))
    cols = min(2, n_plots)
    
    # Clear previous figure
    self.figure.clear()
    
    # Create subplots
    for i, spec in enumerate(valid_specs):
        # Get the plot type and data
        plot_type = spec.get("plot_type", "")
        title = spec.get("title", f"Plot {i+1}")
        x_label = spec.get("x_label", "")
        y_label = spec.get("y_label", "")
        data_spec = spec.get("data", {})
        
        # Create subplot
        ax = self.figure.add_subplot(rows, cols, i+1)
        
        # Create the plot based on type
        if plot_type == "histogram":
            column = data_spec.get("column", "")
            bins = data_spec.get("bins", 30)
            use_log_scale = data_spec.get("use_log_scale", False)
            
            if column in self.sc_analysis.adata.obs:
                sns.histplot(self.sc_analysis.adata.obs[column], bins=bins, kde=False, ax=ax)
                if use_log_scale:
                    ax.set_xscale('log')
            else:
                continue  # Skip this plot, data not available
        
        elif plot_type == "scatter":
            x_column = data_spec.get("x_column", "")
            y_column = data_spec.get("y_column", "")
            
            # Special handling for UMI vs Genes plot
            if "UMI" in title and "Genes" in title:
                # Always use these columns for this specific plot type
                if "total_counts" in self.sc_analysis.adata.obs and "n_genes_by_counts" in self.sc_analysis.adata.obs:
                    x_column = "total_counts"
                    y_column = "n_genes_by_counts"
            
            if x_column in self.sc_analysis.adata.obs and y_column in self.sc_analysis.adata.obs:
                # Make sure data isn't empty and convert to numpy array for plotting
                x_data = self.sc_analysis.adata.obs[x_column].values
                y_data = self.sc_analysis.adata.obs[y_column].values
                
                if len(x_data) > 0 and len(y_data) > 0:
                    # ADD THIS CODE HERE
                    # For large datasets, downsample to improve performance
                    if len(x_data) > 20000:
                        sample_indices = np.random.choice(len(x_data), 20000, replace=False)
                        x_data = x_data[sample_indices]
                        y_data = y_data[sample_indices]
                    # END OF NEW CODE
                    
                    ax.scatter(
                        x_data, y_data,
                        s=3, alpha=0.3
                    )
                    # Always use log scale for UMI counts (total_counts)
                    if x_column == "total_counts" or "UMI" in x_label:
                        ax.set_xscale('log')
                    elif data_spec.get("x_log_scale", False):
                        ax.set_xscale('log')
                    
                    if y_column == "total_counts" or "UMI" in y_label:
                        ax.set_yscale('log')
                    elif data_spec.get("y_log_scale", False):
                        ax.set_yscale('log')
                else:
                 continue  # Skip this plot, data not available
        
        # Add labels
        ax.set_title(title)
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
    
    # Check if any plots were created
    if len(self.figure.axes) > 0:
        # Adjust layout and draw
        self.figure.tight_layout()
        self.canvas.draw()
    else:
        # No valid plots could be created
        self.figure.clear()

# Section 8: Single-cell Analysis Operations: Manages .h5ad loading, dataset summarization, and preprocessing using Scanpy with error handling.
class SingleCellAnalysis:
    """Class for single-cell RNA-seq analysis functions with improved error handling"""
    def __init__(self):
        self.adata = None
        self.file_path = None
        self.last_error = None
        
    def load_h5ad(self, file_path):
        """Load h5ad file into AnnData object"""
        try:
            # Use direct import method without readwrite reference
            import anndata as ad
            self.adata = ad.read_h5ad(file_path)
            self.file_path = file_path
            return f"Successfully loaded {file_path}\nDataset shape: {self.adata.shape[0]} cells x {self.adata.shape[1]} genes"
        except AttributeError as ae:
            if "readwrite" in str(ae):
                # Specific handling for the readwrite error
                try:
                    # Alternative import method for some anndata versions
                    import anndata as ad
                    self.adata = ad.read(file_path)
                    self.file_path = file_path
                    return f"Successfully loaded {file_path}\nDataset shape: {self.adata.shape[0]} cells x {self.adata.shape[1]} genes"
                except Exception as inner_e:
                    return f"Error loading file (alternative method): {str(inner_e)}"
            else:
                return f"Attribute error: {str(ae)}"
        except ImportError:
            return "Error: Required package 'anndata' is not installed. Please install it with 'pip install anndata'."
        except Exception as e:
            return f"Error loading file: {str(e)}"
            
    def get_summary(self) -> str:
        """Get basic summary of the dataset with improved error handling
        
        Returns
        -------
        str
            Summary of the dataset or error message
        """
        if self.adata is None:
            return "No dataset loaded"
        
        try:
            summary = []
            
            # Dataset shape
            summary.append(f"Dataset dimensions: {self.adata.shape[0]} cells x {self.adata.shape[1]} genes")
            
            # Check if X is sparse
            x_type = "sparse" if issparse(self.adata.X) else "dense"
            summary.append(f"Data matrix type: {x_type}")
            
            # Memory usage
            summary.append(f"Approximate memory usage: {self.adata.nbytes / 1e9:.2f} GB")
            
            # Check if data is normalized or scaled
            if hasattr(self.adata, 'uns') and 'log1p' in self.adata.uns:
                summary.append("Data is log-normalized")
                
            # Obs (cell) metadata
            summary.append("\nCell metadata columns:")
            if len(self.adata.obs.columns) == 0:
                summary.append("  - No cell metadata available")
            else:
                for col in self.adata.obs.columns:
                    try:
                        if pd.api.types.is_categorical_dtype(self.adata.obs[col]):
                            # Handle empty categorical data
                            categories = self.adata.obs[col].cat.categories
                            counts = self.adata.obs[col].value_counts()
                            if len(counts) > 0:
                                summary.append(f"  - {col} (categorical): {len(categories)} categories, most common: {counts.index[0]} ({counts.iloc[0]} cells)")
                            else:
                                summary.append(f"  - {col} (categorical): {len(categories)} categories, all empty")
                        else:
                            # For numeric columns, provide statistics
                            if pd.api.types.is_numeric_dtype(self.adata.obs[col]):
                                try:
                                    col_min = self.adata.obs[col].min()
                                    col_max = self.adata.obs[col].max()
                                    summary.append(f"  - {col} (numeric): range [{col_min:.2f} - {col_max:.2f}]")
                                except:
                                    summary.append(f"  - {col} (numeric)")
                            else:
                                summary.append(f"  - {col}")
                    except Exception as e:
                        summary.append(f"  - {col} (error: {str(e)})")
                    
            # Var (gene) metadata
            summary.append("\nGene metadata columns:")
            if len(self.adata.var.columns) == 0:
                summary.append("  - No gene metadata available")
            else:
                for col in self.adata.var.columns:
                    try:
                        if pd.api.types.is_categorical_dtype(self.adata.var[col]):
                            categories = self.adata.var[col].cat.categories
                            counts = self.adata.var[col].value_counts()
                            if len(counts) > 0:
                                summary.append(f"  - {col} (categorical): {len(categories)} categories, most common: {counts.index[0]} ({counts.iloc[0]} genes)")
                            else:
                                summary.append(f"  - {col} (categorical): {len(categories)} categories, all empty")
                        else:
                            # For binary columns like highly_variable, count True values
                            if pd.api.types.is_bool_dtype(self.adata.var[col]):
                                true_count = self.adata.var[col].sum()
                                summary.append(f"  - {col} (boolean): {true_count} True values")
                            else:
                                summary.append(f"  - {col}")
                    except Exception as e:
                        summary.append(f"  - {col} (error: {str(e)})")
            
            # Layers
            if hasattr(self.adata, 'layers') and self.adata.layers:
                summary.append("\nLayers:")
                for layer in self.adata.layers.keys():
                    # Get layer info
                    layer_data = self.adata.layers[layer]
                    layer_type = "sparse" if issparse(layer_data) else "dense"
                    layer_dtype = layer_data.dtype
                    
                    # Get basic stats if possible
                    try:
                        if issparse(layer_data):
                            non_zero = layer_data.nnz
                            sparsity = 1.0 - (non_zero / (layer_data.shape[0] * layer_data.shape[1]))
                            summary.append(f"  - {layer} ({layer_type}, {layer_dtype}, sparsity: {sparsity:.2f})")
                        else:
                            summary.append(f"  - {layer} ({layer_type}, {layer_dtype})")
                    except:
                        summary.append(f"  - {layer}")
            else:
                summary.append("\nLayers: None")
                
            # Check for embeddings like PCA, UMAP, etc.
            if hasattr(self.adata, 'obsm') and len(self.adata.obsm) > 0:
                summary.append("\nEmbeddings:")
                for embed_key in self.adata.obsm.keys():
                    embed_shape = self.adata.obsm[embed_key].shape
                    summary.append(f"  - {embed_key}: {embed_shape[1]} dimensions")
                    
            # Check for graph data
            if hasattr(self.adata, 'obsp') and len(self.adata.obsp) > 0:
                summary.append("\nGraph data:")
                for graph_key in self.adata.obsp.keys():
                    summary.append(f"  - {graph_key}")
            
            return "\n".join(summary)
        except Exception as e:
            return f"Error generating summary: {str(e)}"
    
    def run_basic_preprocessing(self, min_genes: int = 200, min_cells: int = 3, 
                              target_sum: float = 1e4, log_transform: bool = True,
                              find_variable_genes: bool = True) -> str:
        """Run basic scanpy preprocessing with improved error handling and flexibility
        
        Parameters
        ----------
        min_genes : int, default 200
            Minimum number of genes per cell
        min_cells : int, default 3
            Minimum number of cells per gene
        target_sum : float, default 1e4
            Target sum for normalization
        log_transform : bool, default True
            Whether to log-transform the data
        find_variable_genes : bool, default True
            Whether to identify highly variable genes
            
        Returns
        -------
        str
            Success or error message
        """
        if self.adata is None:
            return "No dataset loaded"
        
        try:
            # Store the original dimensions
            original_cells = self.adata.shape[0]
            original_genes = self.adata.shape[1]
            
            # Make a copy to allow recovery if something fails
            original_adata = self.adata.copy()
            
            # Track each step for better error reporting
            current_step = "initial"
            
            try:
                # Calculate QC metrics
                current_step = "QC metrics calculation"
                sc.pp.calculate_qc_metrics(self.adata, inplace=True)
                
                # Basic filtering
                current_step = "cell filtering"
                sc.pp.filter_cells(self.adata, min_genes=min_genes)
                
                current_step = "gene filtering"
                sc.pp.filter_genes(self.adata, min_cells=min_cells)
                
                # Check if we filtered too much
                if self.adata.shape[0] < 10 or self.adata.shape[1] < 10:
                    # Restore original data
                    self.adata = original_adata
                    return (f"WARNING: Filtering removed too many cells or genes. Try lower thresholds.\n"
                           f"Original: {original_cells} cells x {original_genes} genes\n"
                           f"After filtering: {self.adata.shape[0]} cells x {self.adata.shape[1]} genes")
                
                # Normalize
                current_step = "normalization"
                sc.pp.normalize_total(self.adata, target_sum=target_sum)
                
                if log_transform:
                    current_step = "log transformation"
                    sc.pp.log1p(self.adata)
                
                # Find highly variable genes if requested
                if find_variable_genes:
                    current_step = "variable gene detection"
                    # Try to use appropriate parameters based on data
                    try:
                        # Check if the data is suitable for HVG
                        if np.mean(self.adata.X) < 0.01 or np.max(self.adata.X) < 1.0:
                            warnings.warn("Data values are very low; HVG detection may not work well")
                        
                        # Run HVG with appropriate parameters
                        sc.pp.highly_variable_genes(self.adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
                        
                        # Check if we found a reasonable number of HVGs
                        hvg_count = np.sum(self.adata.var['highly_variable'])
                        if hvg_count < 10:
                            # Try again with more permissive parameters
                            sc.pp.highly_variable_genes(self.adata, min_mean=0.01, max_mean=8, min_disp=0.1)
                            hvg_count = np.sum(self.adata.var['highly_variable'])
                            if hvg_count < 10:
                                warnings.warn(f"Found only {hvg_count} highly variable genes. Analysis may be limited.")
                    except Exception as hvg_error:
                        warnings.warn(f"Error in HVG detection: {str(hvg_error)}")
                        # Continue without HVG
                
                # Scale data with care
                current_step = "data scaling"
                # Check if the data is sparse - if so, might need to make it dense
                if issparse(self.adata.X):
                    warnings.warn("Converting sparse data to dense for scaling. This may use significant memory.")
                
                # Cap the maximum values to avoid extreme outliers
                sc.pp.scale(self.adata, max_value=10)
                
                # Calculate success metrics
                cells_kept = self.adata.shape[0]
                genes_kept = self.adata.shape[1]
                cells_pct = (cells_kept / original_cells) * 100
                genes_pct = (genes_kept / original_genes) * 100
                
                return (f"Preprocessing complete.\n"
                       f"Cells: {cells_kept}/{original_cells} ({cells_pct:.1f}%)\n"
                       f"Genes: {genes_kept}/{original_genes} ({genes_pct:.1f}%)\n"
                       f"Processing steps: QC metrics, filtering, normalization{', log transform' if log_transform else ''}"
                       f"{', variable gene identification' if find_variable_genes else ''}, scaling")
                
            except Exception as step_error:
                # Restore original data if any step fails
                self.adata = original_adata
                return f"Error during {current_step}: {str(step_error)}"
               
        except Exception as e:
            return f"Error during preprocessing: {str(e)}"

# Section 9: Helper function to check if a matrix is sparse
def issparse(X):
    """Check if X is a sparse matrix."""
    from scipy import sparse
    return sparse.issparse(X)

# Section 10: Custom text edit widget with fixed text display
class CustomTextEdit(QTextEdit):
    """Custom text edit widget with fixed text display"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptRichText(False)
        
        # Hide scrollbars
        self.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        self.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        
        # Set a fixed height initially
        self.setFixedHeight(35)
        self.line_height = self.fontMetrics().lineSpacing()
        # Connect signals
        self.document().contentsChanged.connect(self.adjust_height)
        
        # Apply styling with adjusted vertical alignment
        self.setStyleSheet("""
            QTextEdit {
                border: none;
                background-color: transparent;
                font-size: 14px;
                padding: 0px;
                margin-top: 0px;
                margin-bottom: 0px;
                color: #1A1A1A;
            }
        """)
        
        # Set document margin to control text placement
        self.document().setDocumentMargin(8)
        
        # Set vertical alignment for text
        self.setAlignment(Qt.AlignVCenter)
        
        # Store line height for calculations
        self.line_height = self.fontMetrics().lineSpacing()
        self.max_visible_lines = 4
        
    def setPlaceholderText(self, text):
        """Override to ensure placeholder text displays correctly"""
        super().setPlaceholderText(text)
        # Force a layout update
        self.document().adjustSize()
        
    def showEvent(self, event):
        """Called when widget becomes visible"""
        super().showEvent(event)
        # Force layout update when shown
        self.document().adjustSize()
        
    def adjust_height(self):
        """Adjust height based on content"""
        # Get current document size
        doc_height = self.document().size().height()
        self.max_visible_lines = 4  
        # Calculate line count
        line_count = max(1, round(doc_height / self.line_height))
        
        # Calculate new height with padding
        if line_count > 1:
            new_height = min(self.line_height * line_count + 12, 
                            self.line_height * self.max_visible_lines + 12)
            self.setFixedHeight(max(35, new_height))
        else:
            # Single line - use fixed height
            self.setFixedHeight(35)
            
    def keyPressEvent(self, event):
        """Handle key press events"""
        if event.key() == Qt.Key_Return and not event.modifiers():
            event.accept()
            # Get main window and call send message
            main_window = self.window()
            if isinstance(main_window, ChatApp):
                main_window.send_message()
                # Reset height
                self.setFixedHeight(35)
        elif event.key() == Qt.Key_Return and event.modifiers() & Qt.ShiftModifier:
            # Shift+Enter inserts newline
            super().keyPressEvent(event)
            self.adjust_height()
        else:
            super().keyPressEvent(event)

# Section 11: Chat Input Widget UI: Manages the chat input area with custom styling and functionality (buttons, send logic, model switching).
class InputWidget(QFrame):
    """Custom input widget with two rows - text input on top, buttons on bottom"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.current_model = "GPTo1"
        self.initUI()

    def initUI(self):
        self.setFrameShape(QFrame.StyledPanel)
        self.setFrameShadow(QFrame.Raised)
        self.setStyleSheet("""
            InputWidget {
            background-color: white;
            border: 1px solid #E5E5E5;
            border-radius: 12px;
            margin: 10px 15px 15px 15px;
            padding: 8px;
        }
        """)

        # Main layout is vertical with two rows
        layout = QVBoxLayout(self)
        layout.setContentsMargins(10, 10, 10, 10)
        layout.setSpacing(8)  # Space between top and bottom rows

        # Top row - only the text input
        self.text_edit = CustomTextEdit(self)
        self.text_edit.setPlaceholderText("Type to talk with Copilot")
        self.text_edit.setStyleSheet("""
        QTextEdit {
            border: none;
            background-color: transparent;
            font-size: 14px;
            padding-top: 1px;  /* Added top padding to prevent text from being cut off */
            color: #1A1A1A;
        }
        """)

        # Bottom row - container for buttons
        buttons_container = QWidget()
        buttons_layout = QHBoxLayout(buttons_container)
        buttons_layout.setContentsMargins(0, 0, 0, 0)
        buttons_layout.setSpacing(8)  # Space between buttons

        # Add plus button
        self.plus_button = QPushButton('+')
        self.plus_button.setFixedSize(26, 30)
        self.plus_button.setStyleSheet("""
            QPushButton {
        background-color: #F5F5F5;
        border: 1px solid #E0E0E0;
        border-radius: 4px;
        color: #333333;
        font-size: 13px;
        padding: 0 8px;
        text-align: left;
            }
            QPushButton:hover {
        background-color: #EAEAEA;
        border-color: #D0D0D0;
            }
        """)

        # Add model button
        self.model_button = QPushButton('GPT o1 ▼')
        self.model_button.setFixedSize(85, 30)
        self.model_button.setStyleSheet("""
            QPushButton {
                background-color: transparent;
                border: none;
                color: #666;
                font-size: 13px;
            }
            QPushButton:hover {
                background-color: #F5F5F5;
                border-radius: 4px;
            }
        """)
        
        # Add microphone button
        self.mic_button = QPushButton()
        self.mic_button.setFixedSize(32, 32)
        self.mic_button.setStyleSheet("""
            QPushButton {
                background-color: transparent;
                border: none;
                color: #666;
                font-size: 16px;
            }
            QPushButton:hover {
                background-color: #F5F5F5;
                border-radius: 4px;
            }
        """)
        self.mic_button.setText("🎤")  # Microphone emoji
        self.mic_button.clicked.connect(self.start_voice_input)

        # Flexible spacer to push submit button to the right
        spacer = QWidget()
        spacer.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Preferred)

        # Submit button
        self.submit_button = QPushButton()
        self.submit_button.setFixedSize(32, 32)
        self.submit_button.setStyleSheet("""
            QPushButton {
                background-color: #3C8D94;
                border: none;
                border-radius: 16px;
                color: white;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #2D6C72;
            }
        """)

        # Use a custom arrow character
        self.submit_button.setText("↑")
        font = QFont()
        font.setPointSize(14)
        font.setBold(True)
        self.submit_button.setFont(font)

        # Add buttons to bottom row
        buttons_layout.addWidget(self.plus_button)
        buttons_layout.addWidget(self.model_button)
        buttons_layout.addWidget(self.mic_button)  # Add the mic button here
        buttons_layout.addWidget(spacer)  # This pushes submit button to the right
        buttons_layout.addWidget(self.submit_button)

        # Add both rows to main layout
        layout.addWidget(self.text_edit)
        layout.addWidget(buttons_container)
        QTimer.singleShot(0, self.text_edit.adjust_height)

    def update_model_button_text(self, model_name):
        """Update the model button text when model changes"""
        self.model_button.setText(f"{model_name} ▼")
        
    def start_voice_input(self):
        """Transcribe speech from microphone and set to input field."""
        parent = self.window()
        
        # Let the user know we're listening
        if hasattr(parent, 'current_chat_index') and parent.current_chat_index >= 0:
            current_chat = parent.chat_history[parent.current_chat_index]
            current_chat["messages"].append({
                "text": "Listening for voice input...",
                "is_user": False
            })
            parent.display_current_chat()
            
            # Change the microphone button style to indicate recording
            original_style = self.mic_button.styleSheet()
            self.mic_button.setStyleSheet("""
                QPushButton {
                    background-color: #FF4444;
                    border: none;
                    border-radius: 16px;
                    color: white;
                    font-size: 16px;
                }
            """)
            
            # Update UI immediately
            QApplication.processEvents()
        
        try:
            # Create a speech recognizer
            r = sr.Recognizer()
            with sr.Microphone() as source:
                # Optional: adjust for ambient noise
                r.adjust_for_ambient_noise(source, duration=1)
                # Listen for speech with a timeout
                audio_data = r.listen(source, timeout=5, phrase_time_limit=10)
            
            # Reset button style before recognition (which might take time)
            self.mic_button.setStyleSheet(original_style)
            QApplication.processEvents()
            
            try:
                # Use Google's free recognition service
                recognized_text = r.recognize_google(audio_data)
                # Put recognized text in the text_edit
                self.text_edit.setPlainText(recognized_text)
                
            except sr.UnknownValueError:
                # Could not understand the audio
                self.text_edit.setPlainText("")
                error_msg = "Sorry, I couldn't understand your speech."
                current_chat["messages"].append({
                    "text": error_msg,
                    "is_user": False
                })
                parent.display_current_chat()
                
            except sr.RequestError as e:
                # Issue with the speech recognition service
                self.text_edit.setPlainText("")
                error_msg = f"Speech recognition service error: {e}"
                current_chat["messages"].append({
                    "text": error_msg,
                    "is_user": False
                })
                parent.display_current_chat()
                
        except Exception as e:
            # Reset button style in case of error
            if 'original_style' in locals():
                self.mic_button.setStyleSheet(original_style)
            
            # General error handling
            error_msg = f"Voice input error: {str(e)}"
            current_chat = parent.chat_history[parent.current_chat_index]
            current_chat["messages"].append({
                "text": error_msg,
                "is_user": False
            })
            parent.display_current_chat()

# Section 12: Main Chat Application
class ChatApp(QMainWindow):
    """Main chat application window"""
    def __init__(self):
        super().__init__()
        # Model configurations
        self.model_configs = {
            "GPTo1": {
                "api_key": "sk-pro",
                "model": "gpt-4",
                "endpoint": "https://api.openai.com/v1/chat/completions"
            },
            "DeepSeekR1": {
                "api_key": "your-deepseek-api-key",
                "model": "deepseek-chat",
                "endpoint": "https://api.deepseek.com/v1/chat/completions"
            },
            "Sonnet": {
                "api_key": "your-anthropic-api-key",
                "model": "claude-3-sonnet-20240229",
                "endpoint": "https://api.anthropic.com/v1/messages"
            }
        }

        self.chat_history = []  # All chat history
        self.current_chat_index = -1  # Current chat index
        self.current_model = "GPTo1"  # Default model
        self.sc_analysis = SingleCellAnalysis()

        self.init_ui()
        self.init_server_connection()
        self.load_chat_history()
        self.new_chat()  # Create a new chat on startup

        # Initialize API client
        self.update_api_config("GPTo1")
        

    def init_ui(self):
        """Initialize user interface"""
        self.setWindowTitle("Bioinformatics Copilot")
        self.setGeometry(100, 100, 1000, 600)
        
        # Set application background color to white and hide scrollbars globally
        self.setStyleSheet("""
            QMainWindow, QWidget {
                background-color: white;
            }
            QScrollBar:vertical, QScrollBar:horizontal {
                width: 0px;
                height: 0px;
                background: transparent;
            }
        """)
        
        # Create main splitter layout
        splitter = QSplitter(Qt.Horizontal)

        # Left side chat history list
        history_panel = QWidget()
        history_panel.setStyleSheet("""
            QWidget {
                 background-color: #F8F9FA;
                 border-right: 1px solid #E5E5E5;
            }
        """)
        history_layout = QVBoxLayout(history_panel)
        self.history_layout = QVBoxLayout(history_panel)

        history_label = QLabel("History")
        history_label.setFont(QFont("Arial", 12, QFont.Bold))
        history_label.setStyleSheet(""" QLabel {
        color: #1A1A1A;
        font-size: 16px;
        padding: 15px 15px 10px 15px;
        border-bottom: 1px solid #E5E5E5;}""")

        self.new_chat_button = QPushButton("New Chat")
        self.new_chat_button.clicked.connect(self.new_chat)
        self.new_chat_button.setStyleSheet("""
            QPushButton {
                background-color: #F2F2F2;
                border: 1px solid #E5E5E5;
                border-radius: 4px;
                padding: 5px;
                color: #333333;
            }
            QPushButton:hover {
                background-color: #E5E5E5;
            }
        """)

        self.history_list = QListWidget()
        self.history_list.itemClicked.connect(self.load_chat)
        self.history_list.setStyleSheet("""
            QListWidget {
                background-color: #F2F2F2;
                border: none;
            }
            QListWidget::item {
                padding: 5px;
                border-bottom: 1px solid #E5E5E5;
                color: #333333;
            }
            QListWidget::item:selected {
                background-color: #DFDFDF;
                color: black;
            }
            QListWidget::item:hover {
                background-color: #E8E8E8;
            }
            QListWidget QScrollBar:vertical {
                width: 0px;
                background: transparent;
            }
        """)
        self.server_button = QPushButton("Connect to Server")
        self.server_button.clicked.connect(self.show_server_login)
        self.server_button.setStyleSheet("""
            QPushButton {
                background-color: #F2F2F2;
                border: 1px solid #E5E5E5;
                border-radius: 4px;
                padding: 5px;
                color: #333333;
            }
            QPushButton:hover {
                background-color: #E5E5E5;
            }
        """)
        # Add it to the history_layout directly
        history_layout.addWidget(self.server_button)
        history_layout.addWidget(history_label)
        history_layout.addWidget(self.new_chat_button)
        history_layout.addWidget(self.history_list)

        # Right side chat area
        chat_panel = QWidget()
        chat_layout = QVBoxLayout(chat_panel)
        chat_layout.setContentsMargins(0, 0, 0, 0)
        chat_layout.setSpacing(5)

        # Use QScrollArea to wrap the chat display area, making it scrollable but hiding scrollbars
        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_area.setFrameShape(QFrame.NoFrame)
        # Hide scrollbars while keeping scrolling functionality
        scroll_area.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        scroll_area.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        scroll_area.setStyleSheet("""
            QScrollArea {
                border: none;
                background-color: white;
            }
            QScrollArea QScrollBar {
                width: 0px;
                height: 0px;
                background: transparent;
            }
        """)

        # Internal container for chat messages
        scroll_content = QWidget()
        scroll_content.setStyleSheet("""
            QWidget {
                background-color: white;
            }
        """)
        scroll_layout = QVBoxLayout(scroll_content)

        # Scrollable chat display
        self.chat_display = QLabel()
        self.chat_display.setWordWrap(True)
        self.chat_display.setOpenExternalLinks(False)  # Make links clickable but not opening externally
        self.chat_display.linkActivated.connect(self.handle_link_click)  # Listen for link clicks
        self.chat_display.setTextInteractionFlags(Qt.TextBrowserInteraction)

        scroll_layout.addWidget(self.chat_display)
        scroll_layout.addStretch()  # Add flexible space to keep messages aligned to the top

        scroll_content.setLayout(scroll_layout)
        scroll_area.setWidget(scroll_content)

        # Add to main layout with stretch factor
        chat_layout.addWidget(scroll_area, 1)  # Set stretch factor to 1 to make display area take more space

        # Custom input widget
        self.input_widget = InputWidget(self)
        self.input_widget.plus_button.clicked.connect(self.show_upload_menu)
        self.input_widget.model_button.clicked.connect(self.show_model_menu)
        self.input_widget.submit_button.clicked.connect(self.send_message)

        chat_layout.addWidget(self.input_widget, 0)  # Set stretch factor to 0 to keep input box at minimum height

        # Add panels to splitter
        splitter.addWidget(history_panel)
        splitter.addWidget(chat_panel)
        splitter.setSizes([200, 800])  # Adjust initial width ratio, giving chat area more space
        splitter.setHandleWidth(1)  # Set separator to thin line
        splitter.setStyleSheet("""
            QSplitter::handle {
                background-color: #E5E5E5;
            }
        """)

        # Set main layout
        container_widget = QWidget()
        main_layout = QVBoxLayout(container_widget)
        main_layout.addWidget(splitter)
        self.setCentralWidget(container_widget)

        # Side Panel (collapsible)
        self.dock = QDockWidget("Side Panel", self)
        self.dock.setAllowedAreas(Qt.LeftDockWidgetArea | Qt.RightDockWidgetArea)
        self.dock_widget = QTextEdit("This is a collapsible panel")
        # Set Side Panel background to white and hide scrollbars
        self.dock_widget.setStyleSheet("""
            QTextEdit {
                background-color: white;
                border: none;
            }
            QTextEdit QScrollBar {
                width: 0px;
                height: 0px;
                background: transparent;
            }
        """)
        self.dock_widget.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        self.dock_widget.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
        self.dock.setWidget(self.dock_widget)
        self.dock.setFloating(False)
        self.addDockWidget(Qt.RightDockWidgetArea, self.dock)
        self.dock.hide()  # Hidden by default

    


    def new_chat(self):
        """Create a new chat"""
        chat = {"title": f"Chat {datetime.now().strftime('%Y-%m-%d %H:%M')}", "messages": []}
        self.chat_history.append(chat)
        self.current_chat_index = len(self.chat_history) - 1
        self.refresh_history_list()
        self.display_current_chat()

    def load_chat(self, item):
        """Load chat history when user clicks on an item in the history list"""
        index = self.history_list.row(item)
        if 0 <= index < len(self.chat_history):
            # Only load the chat if the click wasn't on the delete button
            # (delete button handles its own clicks via connected signal)
            self.current_chat_index = index
            self.display_current_chat()

    def display_current_chat(self):
        """Display current chat content"""
        self.chat_display.clear()
        if self.current_chat_index < 0:
            return

        current_chat = self.chat_history[self.current_chat_index]
        html_content = "<div style='padding: 20px;'>"  # Add padding to entire chat

        for msg in current_chat["messages"]:
            sender = "<b>You</b>" if msg["is_user"] else "<b>Copilot</b>"
            # Improved message bubble styling
            if msg["is_user"]:
                style = """
                    background-color: #F0F7FF;
                    border-radius: 15px 15px 0 15px;
                    margin: 15px 0;
                    padding: 12px 16px;
                    max-width: 80%;
                    float: right;
                    clear: both;
                    box-shadow: 0 1px 2px rgba(0,0,0,0.1);
                """
            else:
                style = """
                    background-color: white;
                    border: 1px solid #E5E5E5;
                    border-radius: 15px 15px 15px 0;
                    margin: 15px 0;
                    padding: 12px 16px;
                    max-width: 80%;
                    float: left;
                    clear: both;
                    box-shadow: 0 1px 2px rgba(0,0,0,0.1);
                """

            # Improved message text styling
            message_text = msg["text"]
            if "side_panel" in msg and msg["side_panel"]:
                message_text += ' <a href="show_panel" style="color: #0066CC; text-decoration: none; margin-left: 8px;">↗ Expand</a>'

            html_content += f"""
            <div style="{style}">
                <div style="color: #666666; font-size: 13px; margin-bottom: 4px;">{sender}</div>
                <div style="color: #1A1A1A; line-height: 1.4;">{message_text.replace('\\n', '<br>')}</div>
            </div>
            <div style="clear: both;"></div>
            """

        html_content += "</div>"
        self.chat_display.setText(html_content)

    
    def delete_chat(self, index):
        """Delete a chat conversation"""
        if 0 <= index < len(self.chat_history):
            # Remove the chat from the history
            del self.chat_history[index]

            # Update current chat index
            if self.current_chat_index == index:
                # If the deleted chat was the current one, switch to another chat
                if len(self.chat_history) > 0:
                    # Switch to the previous chat, or the first one if it was the first
                    self.current_chat_index = max(0, index - 1)
                else:
                    # No chats left
                    self.current_chat_index = -1
                    self.chat_display.clear()
            elif self.current_chat_index > index:
                # If the deleted chat was before the current one, adjust the index
                self.current_chat_index -= 1

            # Refresh the list and display
            self.refresh_history_list()
            if self.current_chat_index >= 0:
                self.display_current_chat()

            # Save changes to file
            self.save_chat_history()
    def send_message(self):
        """Send message from the chat interface"""
        text = self.input_widget.text_edit.toPlainText().strip()
        if not text:
            return

        # Add user message to chat
        if self.current_chat_index < 0:
            self.new_chat()
        current_chat = self.chat_history[self.current_chat_index]
        current_chat["messages"].append({"text": text, "is_user": True})

        # Reset input field
        self.input_widget.text_edit.clear()

        # Display user message first
        self.display_current_chat()
        QApplication.processEvents()  # Update UI

        # Check if this is a chat about analysis
        is_analysis_chat = hasattr(self, 'sc_analysis') and self.sc_analysis.adata is not None

        try:
            if is_analysis_chat:
                # Create analysis context if it doesn't exist
                if not hasattr(self, 'analysis_context'):
                    self.analysis_context = AnalysisContext()
                    # Set basic dataset info
                    self.analysis_context.set_dataset_info({
                        "cells": self.sc_analysis.adata.shape[0],
                        "genes": self.sc_analysis.adata.shape[1],
                        "filename": os.path.basename(self.sc_analysis.file_path)
                    })

                # Handle as analysis chat
                if not hasattr(self, 'ai_client'):
                    self.ai_client = AIAnalysisClient(api_key=self.model_configs["GPTo1"]["api_key"])

                # Share analysis context with AI client
                self.ai_client.analysis_context = self.analysis_context

                # Get AI response about analysis
                ai_response = self.ai_client.handle_analysis_chat(text)
            else:
                # Regular chat (your existing code)
                ai_response = "This is a regular chat response. To discuss analysis, please load a dataset first."

            # Add AI response
            current_chat["messages"].append({
                "text": ai_response,
                "is_user": False
            })
        except Exception as e:
            # Add error message
            current_chat["messages"].append({
                "text": f"Error: {str(e)}",
                "is_user": False
            })

        self.refresh_history_list()
        self.display_current_chat()

    def refresh_history_list(self):
     """Refresh chat history list with added delete buttons"""
     self.history_list.clear()
     for i, chat in enumerate(self.chat_history):
        # Create custom widget for each item
        item_widget = QWidget()
        layout = QHBoxLayout(item_widget)
        layout.setContentsMargins(5, 2, 5, 2)
        
        # Chat title label
        title_label = QLabel(chat["title"])
        title_label.setStyleSheet("color: #333333;")
        
        # Delete button (trash icon)
        delete_button = QPushButton()
        delete_button.setFixedSize(24, 24)
        delete_button.setStyleSheet("""
            QPushButton {
                background-color: transparent;
                border: none;
                color: #888888;
                font-size: 14px;
            }
            QPushButton:hover {
                color: #FF5555;
            }
        """)
        delete_button.setText("🗑️")  # Trash bin emoji
        
        # Connect delete button to function with the specific chat index
        delete_button.clicked.connect(lambda checked, idx=i: self.delete_chat(idx))
        
        # Add widgets to layout
        layout.addWidget(title_label, 1)  # 1 = stretch factor to take available space
        layout.addWidget(delete_button, 0)  # 0 = fixed size
        
        # Create list item and set custom widget
        item = QListWidgetItem()
        # Set size hint to accommodate the custom widget
        item.setSizeHint(item_widget.sizeHint())
        self.history_list.addItem(item)
        self.history_list.setItemWidget(item, item_widget)

    # Select current chat
     if self.current_chat_index >= 0 and self.current_chat_index < self.history_list.count():
         self.history_list.setCurrentRow(self.current_chat_index)

    def init_server_connection(self):
        """Initialize server connection components"""
        # Add a server connection button to your UI
        self.server_button = QPushButton("Connect to Server")
        self.server_button.clicked.connect(self.show_server_login)
        self.server_button.setStyleSheet("""
            QPushButton {
                background-color: #F2F2F2;
                border: 1px solid #E5E5E5;
                border-radius: 4px;
                padding: 5px;
                color: #333333;
            }
            QPushButton:hover {
                background-color: #E5E5E5;
            }
        """)

        # Add this button to your UI - for example, to the history panel
        # Assuming history_layout is your QVBoxLayout for history panel
        # Add this after the new_chat_button but before the history_list
        self.history_layout.addWidget(self.server_button)

        # Initialize SSH client
        self.ssh_client = MySSHClient()
        self.server_connected = False

    def show_server_login(self):
        """Show server login dialog"""
        login_dialog = ServerLoginBox()
        if login_dialog.exec_() == QDialog.Accepted:
            username = login_dialog.username
            host = login_dialog.host
            password = login_dialog.password

            # Connect to server
            self.connect_to_server(username, host, password)
    
    def connect_to_server(self, username, host, password):
        """Connect to remote server via SSH"""
        # Show connecting message in chat
        if self.current_chat_index < 0:
            self.new_chat()
        current_chat = self.chat_history[self.current_chat_index]
        current_chat["messages"].append({
            "text": f"Connecting to server {host}...",
            "is_user": False
        })
        self.display_current_chat()
        QApplication.processEvents()  # Update UI

        # Try to connect
        success, message = self.ssh_client.connect_to_server(host, username, password)

        # Add result message to chat
        current_chat["messages"].append({
            "text": message,
            "is_user": False
        })
        self.display_current_chat()

        # Update server connection status
        self.server_connected = success

        # Update button text to show connection status
        if success:
            self.server_button.setText(f"Connected to {host}")
            self.server_button.setStyleSheet("""
                QPushButton {
                    background-color: #E6F4EA;
                    border: 1px solid #34A853;
                    border-radius: 4px;
                    padding: 5px;
                    color: #34A853;
                }
            """)

            # List available datasets on server
            self.list_server_datasets()
        else:
            self.server_button.setText("Connect to Server")

    def list_server_datasets(self):
        """List available datasets on the server"""
        if not self.server_connected:
            return

        # Execute command to list h5ad files in a common data directory
        success, output = self.ssh_client.execute_command("find /path/to/data -name '*.h5ad'")

        if success:
            # Parse output and show in chat
            datasets = output.strip().split('\n')

            if datasets and datasets[0]:  # Check if list is not empty and first item is not empty
                current_chat = self.chat_history[self.current_chat_index]
                message = "Available datasets on server:\n"

                for idx, dataset in enumerate(datasets):
                    # Create clickable links for datasets
                    dataset_name = os.path.basename(dataset)
                    message += f"{idx+1}. <a href='dataset:{dataset}'>{dataset_name}</a>\n"

                current_chat["messages"].append({
                    "text": message,
                    "is_user": False
                })
                self.display_current_chat()
            else:
                # No datasets found
                current_chat = self.chat_history[self.current_chat_index]
                current_chat["messages"].append({
                    "text": "No datasets found on server.",
                    "is_user": False
                })
                self.display_current_chat()
        else:
            # Command failed
            current_chat = self.chat_history[self.current_chat_index]
            current_chat["messages"].append({
                "text": f"Failed to list datasets: {output}",
                "is_user": False
            })
            self.display_current_chat()
    
    def download_server_dataset(self, remote_path):
        """Download a dataset from the server"""
        if not self.server_connected:
            return

        # Create local directory for downloaded files if it doesn't exist
        local_dir = os.path.join(os.path.expanduser("~"), "bioinformatics_copilot", "data")
        os.makedirs(local_dir, exist_ok=True)

        # Get the filename from the path
        filename = os.path.basename(remote_path)
        local_path = os.path.join(local_dir, filename)

        # Show downloading message
        current_chat = self.chat_history[self.current_chat_index]
        current_chat["messages"].append({
            "text": f"Downloading {filename}...",
            "is_user": False
        })
        self.display_current_chat()
        QApplication.processEvents()  # Update UI

        # Download the file
        success, message = self.ssh_client.download_file(remote_path, local_path)

        # Show result message
        current_chat["messages"].append({
            "text": message,
            "is_user": False
        })
        self.display_current_chat()

        # If download was successful, load the file
        if success:
            self.handle_h5ad_file(local_path)


    def resize_dock(self):
        """Make Side Panel take half of the window size"""
        total_width = self.width()
        self.resizeDocks([self.dock], [total_width // 2], Qt.Horizontal)

    def show_dock(self):
        """Show Side Panel and adjust size"""
        self.dock.show()
        self.resize_dock()

    def handle_link_click(self, url):
        """Handle hyperlink click"""
        if url == "show_panel":
            self.show_dock()
        elif url.startswith("dataset:"):
            # Extract the remote path from the URL
            remote_path = url[8:]  # Remove "dataset:" prefix
            self.download_server_dataset(remote_path)

    def save_chat_history(self):
        """Save chat history"""
        with open("chat_history.json", "w", encoding="utf-8") as f:
            json.dump(self.chat_history, f, ensure_ascii=False, indent=2)

    def load_chat_history(self):
        """Load chat history"""
        if os.path.exists("chat_history.json"):
            with open("chat_history.json", "r", encoding="utf-8") as f:
                self.chat_history = json.load(f)
            self.refresh_history_list()

    def update_api_config(self, model_name):
        """Update API configuration when model changes"""
        config = self.model_configs[model_name]
        self.current_model = model_name
        self.input_widget.update_model_button_text(model_name)

        if model_name == "GPTo1":
            self.openai_client = openai.OpenAI(api_key=config["api_key"])
        # Implement other clients here

    def show_model_menu(self):
        """Show model selection menu"""
        menu = QMenu(self)
        menu.setStyleSheet("""
            QMenu {
            background-color: white;
            border: 1px solid #E0E0E0;
            border-radius: 6px;
            padding: 4px 0;
        }
        QMenu::item {
            padding: 8px 24px 8px 8px;
            color: #333333;
            font-size: 13px;
        }
        QMenu::item:selected {
            background-color: #F0F7FF;
        }
        QMenu::indicator {
            width: 18px;
            height: 18px;
            padding-left: 6px;
        }
    """)


        for model_name in self.model_configs.keys():
            action = QAction(model_name, self)
            action.triggered.connect(lambda checked, m=model_name: self.update_api_config(m))
            if model_name == self.current_model:
                action.setIcon(self.style().standardIcon(QStyle.SP_DialogApplyButton))
            
            menu.addAction(action)

        # Position menu below the button
        button_pos = self.input_widget.model_button.mapToGlobal(
            self.input_widget.model_button.rect().bottomLeft()
        )
        menu.exec_(button_pos)

    def show_upload_menu(self):
        """Show upload options menu"""
        menu = QMenu(self)
        menu.setStyleSheet("""
            QMenu {
                background-color: white;
                border: 1px solid #E5E5E5;
                border-radius: 4px;
            }
            QMenu::item {
                padding: 8px 20px;
                color: #000000;  /* Add this line to set text color for all menu items */
            }
            QMenu::item:selected {
                background-color: #F0F0F0;
                 color: #000000;
            }
        """)

        actions = [
            ("Upload File", self.show_file_dialog),
            ("Upload Photo", self.upload_photo),
            ("Take Screenshot", self.take_screenshot),
            ("Take Photo", self.take_photo),
            ("Search the Web", self.search_web)
        ]

        for text, slot in actions:
            action = QAction(text, self)
            action.triggered.connect(slot)
            menu.addAction(action)

        button_pos = self.input_widget.plus_button.mapToGlobal(
            self.input_widget.plus_button.rect().bottomLeft()
        )
        menu.exec_(button_pos)
    def setup_single_cell_panel(self):
     """Set up the side panel for single cell analysis"""
     # Make sure we have the necessary imports
     try:
         import matplotlib
         matplotlib.use('Qt5Agg')  # Set backend
         from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
         from matplotlib.figure import Figure
     except ImportError:
         # Handle missing dependencies
         if not hasattr(self, 'dock') or self.dock is None:
             self.dock = QDockWidget("Single Cell Analysis", self)
             self.dock.setAllowedAreas(Qt.RightDockWidgetArea)
             self.addDockWidget(Qt.RightDockWidgetArea, self.dock)

         error_widget = QWidget()
         error_layout = QVBoxLayout(error_widget)
         error_label = QLabel("Missing dependencies for single cell analysis.")
         install_label = QLabel("Please install required packages:\npip install anndata scanpy matplotlib seaborn")
         error_layout.addWidget(error_label)
         error_layout.addWidget(install_label)
         self.dock.setWidget(error_widget)
         return

     # Create panel
     sc_widget = QWidget()
     sc_layout = QVBoxLayout(sc_widget)

     # Dataset info
     file_name = os.path.basename(self.sc_analysis.file_path) if self.sc_analysis.file_path else "No file"
     file_label = QLabel(f"Dataset: {file_name}")
     file_label.setStyleSheet("font-weight: bold; color: black;")
     sc_layout.addWidget(file_label)

     if self.sc_analysis.adata is not None:
         info_label = QLabel(f"Cells: {self.sc_analysis.adata.shape[0]} | Genes: {self.sc_analysis.adata.shape[1]}")
         info_label.setStyleSheet("color: black;")
         sc_layout.addWidget(info_label)

     # Analysis options
     analysis_label = QLabel("Analysis:")
     analysis_label.setStyleSheet("color: black;")
     sc_layout.addWidget(analysis_label)

     self.analysis_combo = QComboBox()
     self.analysis_combo.addItems([
         "Dataset Summary", 
         "Quality Control", 
         "Preprocessing", 
         "PCA Visualization",
         "UMAP Visualization", 
         "Clustering", 
         "Marker Genes",
         "Heatmap",
         "KEGG Pathway Analysis"
     ])
     self.analysis_combo.setStyleSheet("""
         QComboBox {
             color: black;
             background-color: white;
             border: 1px solid #E0E0E0;
             border-radius: 4px;
             padding: 4px 8px;
             min-height: 25px;
             font-size: 13px;
         }
         QComboBox:hover {
             border-color: #CCCCCC;
         }
         QComboBox::drop-down {
             subcontrol-origin: padding;
             subcontrol-position: center right;
             width: 20px;
             border-left: 1px solid #E0E0E0;
         }
         QComboBox QAbstractItemView {
             color: black;
             background-color: white;
             selection-background-color: #F0F7FF;
             selection-color: black;
             border: 1px solid #E0E0E0;
         }
     """)
     sc_layout.addWidget(self.analysis_combo)

     # Run button
     run_button = QPushButton("Run Analysis")
     run_button.setStyleSheet("""
         QPushButton {
             background-color: #3C8D94;
             color: white;
             border: none;
             border-radius: 4px;
             padding: 6px 12px;
             font-weight: bold;
         }
         QPushButton:hover {
             background-color: #2D6C72;
         }
     """)
     run_button.clicked.connect(self.run_single_cell_analysis)
     sc_layout.addWidget(run_button)

     # Results area
     results_label = QLabel("Results:")
     results_label.setStyleSheet("color: black;")
     sc_layout.addWidget(results_label)

     # Text output - use a consistent name
     self.sc_result_text = QTextEdit()
     self.sc_result_text.setReadOnly(True)
     self.sc_result_text.setMinimumHeight(150)
     self.sc_result_text.setStyleSheet("""
         QTextEdit {
             background-color: white;
             color: black;
             border: 1px solid #E0E0E0;
             font-family: monospace;
             font-size: 13px;
             padding: 8px;
         }
     """)
     sc_layout.addWidget(self.sc_result_text)

     # Visualization area
     vis_label = QLabel("Visualization:")
     vis_label.setStyleSheet("color: black;")
     sc_layout.addWidget(vis_label)

     # Figure canvas for plots
     self.figure = Figure(figsize=(6, 5))
     self.canvas = FigureCanvas(self.figure)
     self.canvas.setMinimumHeight(300)
     sc_layout.addWidget(self.canvas)

     # Create dock if it doesn't exist
     if not hasattr(self, 'dock') or self.dock is None:
         self.dock = QDockWidget("Single Cell Analysis", self)
         self.dock.setAllowedAreas(Qt.RightDockWidgetArea)
         self.addDockWidget(Qt.RightDockWidgetArea, self.dock)

     # Set the widget
     self.dock.setWidget(sc_widget)
     self.dock.setWindowTitle("Single Cell Analysis")
    def show_file_dialog(self):
        """Show file selection dialog"""
        file_dialog = QFileDialog(self)
        file_dialog.setFileMode(QFileDialog.ExistingFile)
        file_dialog.setNameFilter("All Files (*);;h5ad Files (*.h5ad)")
        file_dialog.setViewMode(QFileDialog.Detail)
        if file_dialog.exec_():
            selected_files = file_dialog.selectedFiles()
            if selected_files:
                file_path = selected_files[0]
                file_name = os.path.basename(file_path)

                # Check if it's an h5ad file
                if file_path.lower().endswith('.h5ad'):
                    # Add special handling for h5ad files
                    self.handle_h5ad_file(file_path)
                else:
                    # Regular file handling
                    self.input_widget.text_edit.append(f"[File: {file_name}]")
    def handle_h5ad_file(self, file_path):
        """Special handling for h5ad files"""
        try:
            # Let the user know the file is being processed
            file_name = os.path.basename(file_path)
            self.input_widget.text_edit.append(f"[Single Cell Data: {file_name}]")

            # Initialize SingleCellAnalysis if not already done
            if not hasattr(self, 'sc_analysis'):
                self.sc_analysis = SingleCellAnalysis()

            # Load the file and add a system message about it
            result = self.sc_analysis.load_h5ad(file_path)

            # Create a special message in the chat
            current_chat = self.chat_history[self.current_chat_index]
            current_chat["messages"].append({
                "text": f"Loaded single-cell data file: {file_name}\n\n{result}",
                "is_user": False,
                "side_panel": True  # Enable side panel button
            })

            # Show the side panel with analysis options
            self.setup_single_cell_panel()
            self.display_current_chat()
            self.show_dock()

        except Exception as e:
            # Add error message to chat
            current_chat = self.chat_history[self.current_chat_index]
            current_chat["messages"].append({
                "text": f"Error loading {file_name}: {str(e)}",
                "is_user": False
            })
            self.display_current_chat()

    def upload_photo(self):
        """Upload photo functionality"""
        # Placeholder for future implementation
        self.input_widget.text_edit.append("[Upload Photo selected]")

    

    def load_single_cell_file(self):
        """Show file selection dialog for h5ad files"""
        file_dialog = QFileDialog(self)
        file_dialog.setFileMode(QFileDialog.ExistingFile)
        file_dialog.setNameFilter("h5ad Files (*.h5ad)")
        if file_dialog.exec_():
            selected_files = file_dialog.selectedFiles()
            if selected_files:
                file_path = selected_files[0]
                result = self.sc_analysis.load_h5ad(file_path)
                self.file_label.setText(os.path.basename(file_path))
                self.results_text.setText(result)
    
    def run_single_cell_analysis(self):
        """Run the selected single cell analysis using AI agents"""
        if not hasattr(self, 'sc_analysis') or self.sc_analysis.adata is None:
            if hasattr(self, 'sc_result_text'):
                self.sc_result_text.setText("No dataset loaded. Please load an .h5ad file first.")
            return

        analysis_type = self.analysis_combo.currentText()
        self.sc_result_text.setText(f"Running {analysis_type} with AI assistant...")
        QApplication.processEvents()  # Update UI

        try:
            # Ensure appropriate preprocessing is done based on analysis type
            if analysis_type in ["Quality Control", "Preprocessing"]:
                # Make sure QC metrics are calculated if not already there
                if 'n_genes_by_counts' not in self.sc_analysis.adata.obs:
                    self.sc_result_text.setText("Calculating QC metrics...")
                    QApplication.processEvents()
                    sc.pp.calculate_qc_metrics(self.sc_analysis.adata, inplace=True)

            elif analysis_type in ["PCA Visualization"]:
                # Check if preprocessing is done
                if 'highly_variable' not in self.sc_analysis.adata.var:
                    self.sc_result_text.setText("Running preprocessing before PCA...")
                    QApplication.processEvents()

                    # Run basic preprocessing
                    sc.pp.normalize_total(self.sc_analysis.adata, target_sum=1e4)
                    sc.pp.log1p(self.sc_analysis.adata)
                    sc.pp.highly_variable_genes(self.sc_analysis.adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

                # Run PCA if needed
                if 'X_pca' not in self.sc_analysis.adata.obsm:
                    self.sc_result_text.setText("Calculating PCA...")
                    QApplication.processEvents()
                    sc.pp.pca(self.sc_analysis.adata, svd_solver='arpack')

            elif analysis_type in ["UMAP Visualization", "Clustering"]:
                # Check if PCA is done
                if 'X_pca' not in self.sc_analysis.adata.obsm:
                    self.sc_result_text.setText("Running PCA before UMAP...")
                    QApplication.processEvents()

                    # Run basic preprocessing if not done
                    if 'highly_variable' not in self.sc_analysis.adata.var:
                        sc.pp.normalize_total(self.sc_analysis.adata, target_sum=1e4)
                        sc.pp.log1p(self.sc_analysis.adata)
                        sc.pp.highly_variable_genes(self.sc_analysis.adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

                    sc.pp.pca(self.sc_analysis.adata, svd_solver='arpack')

                # Run UMAP if needed
                if 'X_umap' not in self.sc_analysis.adata.obsm:
                    self.sc_result_text.setText("Calculating UMAP...")
                    QApplication.processEvents()
                    sc.pp.neighbors(self.sc_analysis.adata, n_neighbors=15, n_pcs=30)
                    sc.tl.umap(self.sc_analysis.adata)

                # For clustering, run leiden if needed
                if analysis_type == "Clustering" and 'leiden' not in self.sc_analysis.adata.obs:
                    self.sc_result_text.setText("Running clustering...")
                    QApplication.processEvents()
                    sc.tl.leiden(self.sc_analysis.adata, resolution=0.5)

            elif analysis_type == "Marker Genes":
                # Check if clustering is done
                if 'leiden' not in self.sc_analysis.adata.obs:
                    self.sc_result_text.setText("Running clustering before marker gene analysis...")
                    QApplication.processEvents()

                    # Run prerequisites if needed
                    if 'X_pca' not in self.sc_analysis.adata.obsm:
                        # Run preprocessing if not done
                        if 'highly_variable' not in self.sc_analysis.adata.var:
                            sc.pp.normalize_total(self.sc_analysis.adata, target_sum=1e4)
                            sc.pp.log1p(self.sc_analysis.adata)
                            sc.pp.highly_variable_genes(self.sc_analysis.adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

                        sc.pp.pca(self.sc_analysis.adata, svd_solver='arpack')

                    if 'X_umap' not in self.sc_analysis.adata.obsm:
                        sc.pp.neighbors(self.sc_analysis.adata, n_neighbors=15, n_pcs=30)
                        sc.tl.umap(self.sc_analysis.adata)

                    sc.tl.leiden(self.sc_analysis.adata, resolution=0.5)

                # Calculate marker genes if not already done
                if not hasattr(self, 'marker_genes_calculated'):
                    self.sc_result_text.setText("Calculating marker genes...")
                    QApplication.processEvents()
                    sc.tl.rank_genes_groups(self.sc_analysis.adata, 'leiden', method='wilcoxon')
                    self.marker_genes_calculated = True

            # Prepare data summary for the agent
            data_summary = prepare_data_summary(self)

            # Initialize AI client if not already done
            if not hasattr(self, 'ai_client'):
                self.ai_client = AIAnalysisClient(api_key=self.model_configs["GPTo1"]["api_key"])

            # Run analysis using AI agent
            result = self.ai_client.run_analysis(analysis_type, data_summary)

            # Create or update analysis context
            if not hasattr(self, 'analysis_context'):
                self.analysis_context = AnalysisContext()
                # Set basic dataset info
                self.analysis_context.set_dataset_info({
                    "cells": self.sc_analysis.adata.shape[0],
                    "genes": self.sc_analysis.adata.shape[1],
                    "filename": os.path.basename(self.sc_analysis.file_path)
                })

            # Update analysis context with results
            self.analysis_context.update_from_analysis(
                analysis_type,
                result.get('text_summary', ''),
                result.get('recommendations', {})
            )

            # Check for error
            if 'error' in result:
                error_msg = f"Error during analysis: {result['error']}"
                self.sc_result_text.setText(error_msg)
                print(error_msg)
                return

            # Update text results using text_summary if available
            if 'text_summary' in result:
                self.sc_result_text.setText(result['text_summary'])
            else:
                self.sc_result_text.setText("Analysis completed, but no detailed summary was provided.")

            # Clear previous figure
            self.figure.clear()

            # Create visualization based on agent response if visualization_specs exists
            if 'visualization_specs' in result and result['visualization_specs']:
                create_visualization_from_agent_result(self, result['visualization_specs'])
            else:
                # No visualization was generated, draw empty figure
                self.canvas.draw()

        except Exception as e:
            error_msg = f"Error during analysis: {str(e)}"
            self.sc_result_text.setText(error_msg)
            import traceback
            print(traceback.format_exc())


    def take_screenshot(self):
                """Take screenshot functionality"""
                # Placeholder for future implementation
                self.input_widget.text_edit.append("[Screenshot requested]")

    def take_photo(self):
        """Take photo functionality"""
        # Placeholder for future implementation
        self.input_widget.text_edit.append("[Take Photo selected]")

    def search_web(self):
        """Search web functionality"""
        # Placeholder for future implementation
        self.input_widget.text_edit.append("[Web Search requested]")

    def closeEvent(self, event):
        """Handle window close event"""
        self.save_chat_history()
        event.accept()

# Section 13: Application Entry Point
if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.setStyle('Fusion')  # For consistent styling across platforms
    window = ChatApp()
    window.show()
    sys.exit(app.exec_())

/opt/anaconda3/envs/bioinformaticscopilot/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/anaconda3/envs/bioinformaticscopilot/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout


Could not import the PyAudio C module 'pyaudio._portaudio'.
Could not import the PyAudio C module 'pyaudio._portaudio'.


2025-04-13 13:08:41.150 python[11751:510339] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
